Análises estatísticas e Gráficos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
sns.set_theme(style="whitegrid", palette="viridis")
plt.rcParams['figure.figsize'] = (16, 10)
plt.rcParams['axes.titlesize'] = 14

In [3]:
path_df = "../data/processed/dataset_not_null_features_removed_outliers"

In [4]:
df = pd.read_csv(path_df)

In [5]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,member_casual,round_trip,ride_duration,ride_duration_minutes,week_day,is_weekend,hour_start,hour_end,distance_km,distance_m
0,322BD23D287743ED,docked_bike,2020-08-20 18:08:14,2020-08-20 18:17:51,Lake Shore Dr & Diversey Pkwy,329.0,Clark St & Lincoln Ave,141.0,41.932588,-87.636427,...,member,different_place,0 days 00:09:37,9.616667,Thursday,False,18,18,1.885153,1885.152542
1,2A3AEF1AB9054D8B,electric_bike,2020-08-27 18:46:04,2020-08-27 19:54:51,Michigan Ave & 14th St,168.0,Michigan Ave & 14th St,168.0,41.864379,-87.623681,...,casual,same_place,0 days 01:08:47,68.783333,Thursday,False,18,19,0.026647,26.647146
2,13814D3D661ECADB,electric_bike,2020-08-27 16:49:02,2020-08-27 16:59:49,Leavitt St & Division St,658.0,Leavitt St & Division St,658.0,41.902989,-87.683767,...,casual,same_place,0 days 00:10:47,10.783333,Thursday,False,16,16,0.006598,6.597641
3,56349A5A42F0AE51,electric_bike,2020-08-27 17:26:23,2020-08-27 18:07:50,Leavitt St & Division St,658.0,Leavitt St & Division St,658.0,41.903015,-87.683731,...,casual,same_place,0 days 00:41:27,41.450000,Thursday,False,17,18,0.012079,12.079038
4,EB6ABC5570C29B22,electric_bike,2020-08-26 20:14:02,2020-08-26 20:34:00,Cityfront Plaza Dr & Pioneer Ct,196.0,Dearborn St & Monroe St,49.0,41.890326,-87.621700,...,casual,different_place,0 days 00:19:58,19.966667,Wednesday,False,20,20,1.250438,1250.438215


In [9]:
def analise_estatistica_completa(df):
    print("=== 📊 SUMMARY ===")
    print(f"Lines: {df.shape[0]}")
    print(f"Columns: {df.shape[1]}\n")

    # 1. Verifing null values
    print("=== 🔍 NULL VALUES ===")
    nulls = df.isnull().sum()
    print(nulls[nulls > 0] if nulls.any() else "Any null value founded.", "\n")

    # 2. Numerical Variables Analysis
    # (distance, duration, hours)
    cols_num = ['ride_duration_minutes', 'distance_km', 'distance_m', 'hour_start', 'hour_end']
    print("=== 📈 NUMERICAL STATISTICS ===")
    resumo_num = df[cols_num].describe().T
    # Adding median
    resumo_num['median'] = df[cols_num].median()
    print(resumo_num[['count', 'mean', 'std', 'min', 'median', 'max']], "\n")

    # 3. Categorical Variable Analysis
    cols_cat = ['rideable_type', 'member_casual', 'start_station_name', 'end_station_name', 'week_day','round_trip']
    print("=== 🔠 CATEGORICAL ANALYSIS (Top 3 Values) ===")
    for col in cols_cat:
        top_values = df[col].value_counts().head(3)
        percent = (top_values / len(df) * 100).round(2)
        print(f"Column: {col}")
        for val, count in top_values.items():
            print(f"  - {val}: {count} ({percent[val]}%)")
    print("")

    # 4. Bools Variable Analysis
    cols_bool = ['is_weekend']
    print("=== 🚩 BOOLS DISTRIBUTION ===")
    for col in cols_bool:
        dist = df[col].value_counts(normalize=True) * 100
        print(f"{col}: {dist.get(True, 0):.2f}% Sim | {dist.get(False, 0):.2f}% Não")
    print("")

    # 5. Simple Detection Outliers (Using IQR for distance and time)
    print("=== 🚨 OUTLIERS (IQR) ===")
    for col in ['ride_duration_minutes', 'distance_km']:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        limite_superior = Q3 + 1.5 * IQR
        outliers = df[df[col] > limite_superior].shape[0]
        print(f"{col}: {outliers} lines above {limite_superior:.2f} (Superior Limit)")
    print("")
    
    print("*" * 70)

    # 6. Business Insight: round_trip and duration/distance
    print("\n🎯 INSIGHT: BEHAVIOR FOR ROUND TRIP (Mean):")
    # Agrupa por round_trip para ver a diferença de duração e distância
    insight_trip = df.groupby('round_trip')[['ride_duration_minutes', 'distance_km']].mean().round(2)
    print(insight_trip)
    print("")

    # 7. Business Insight: round_trip and duration/distance
    print("\n🎯 INSIGHT: MEMBER FOR ROUND TRIP (Mean):")
    # Agrupa por round_trip para ver a diferença de duração e distância
    insight_trip = df.groupby('member_casual')[['ride_duration_minutes', 'distance_km']].mean().round(2)
    print(insight_trip)
    print("")

In [10]:
analise_estatistica_completa(df)

=== 📊 SUMMARY ===
Lines: 523479
Columns: 22

=== 🔍 NULL VALUES ===
distance_km    835
distance_m     835
dtype: int64 

=== 📈 NUMERICAL STATISTICS ===
                          count         mean          std  min       median  \
ride_duration_minutes  523479.0    18.910544    17.133092  0.0    14.516667   
distance_km            522644.0     1.957801     1.502788  0.0     1.647392   
distance_m             522644.0  1957.801116  1502.788477  0.0  1647.391624   
hour_start             523479.0    14.427033     4.707213  0.0    15.000000   
hour_end               523479.0    14.607335     4.802208  0.0    15.000000   

                               max  
ride_duration_minutes   147.000000  
distance_km               6.759760  
distance_m             6759.759605  
hour_start               23.000000  
hour_end                 23.000000   

=== 🔠 CATEGORICAL ANALYSIS (Top 3 Values) ===
Column: rideable_type
  - docked_bike: 477087 (91.14%)
  - electric_bike: 46392 (8.86%)
Column: member_c